In [32]:
import pandas as pd
import numpy as np
from itertools import product

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import cross_val_score, train_test_split
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

In [21]:
df = pd.read_csv('train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [22]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)

In [23]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [24]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
classifier = Pipeline([('comment_text_selector', FeatureSelector(column='comment_text')), 
                     ('comment_text_tfidf', TfidfVectorizer(sublinear_tf=True,
                                                            strip_accents='unicode',
                                                            analyzer='word',
                                                            token_pattern=r'\w{1,}',
                                                            stop_words='english',
                                                            ngram_range=(1, 1),
                                                            max_features=10000)), 
                     ('clf', LogisticRegression(C=0.1, solver='sag'))])

In [25]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

In [45]:
def result_df():
    models = [LogisticRegression(random_state = 42), KNeighborsClassifier(), RandomForestClassifier(), GradientBoostingClassifier()]
    metrics = ['roc_auc', 'precision', 'recall', 'f1']
    n = 0
    res = []
    for model, metric in product(models, metrics):
        n += 1 
        classifier = Pipeline([
            ('features',feats),
            ('classifier', model),
        ])
        cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring=f'{metric}')
        cv_score = np.mean(cv_scores)
        cv_score_std = np.std(cv_scores)
        res.append([cv_score, cv_score_std])
        print(n)
    return res

In [46]:
result = result_df()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [58]:
resultx

[0.7867401104915408,
 0.7458662096326989,
 0.6786941170459948,
 0.7106367047798583,
 0.6919218138274713,
 0.6557027441597153,
 0.628539147315234,
 0.6417753660050671,
 0.7740219475169634,
 0.7187105653646337,
 0.7038280489890756,
 0.7089289266316413,
 0.8025108960590339,
 0.7560700476243205,
 0.6969075377662072,
 0.7252300244017327]

In [54]:
resultx = [i[0]for i in result]
x = []
z = []
n = 0
for i in resultx:
    n += 1
    z.append(i)
    if (n % 4) == 0:
        x.append(z)
        z = []

In [59]:
df_result = pd.DataFrame(data=x, columns=['roc_auc', 'precision', 'recall', 'f1'], index=['LogisticRegression','KNeighbors','RandomForest','GradientBoosting'])

In [60]:
df_result

,roc_auc,precision,recall,f1
LogisticRegression,0.786740,0.745866,0.678694,0.710637
KNeighbors,0.691922,0.655703,0.628539,0.641775
RandomForest,0.774022,0.718711,0.703828,0.708929
GradientBoosting,0.802511,0.756070,0.696908,0.725230


Ну как видно если считать что-то вроде среднего от всех метрик, лучше всего справился градиентный бустинг. Хотя тут не у одной модели не настраивались параметры, да и в целом стоит оценивать модели и добиваться лучшего качества только по одной из метрик, так что это сравнение некорректно.